## Tutorial on Spectral semi-sprays (specs)

---

In [ ]:
import casadi as ca
import numpy as np
from fabrics.diffGeometry.spec import Spec

#### 1. Simple spec

A Spectral semi-spray (spec) is a pair $\left(\mathbf{M}(\mathbf{x}, \mathbf{\dot{x}}), \mathbf{f}(\mathbf{x}, \mathbf{\dot{x}})\right)$ representing a differential equation of the form $\mathbf{M}\ddot{\mathbf{x}} + \mathbf{f} = \mathbf{0}$.\
We will create a very simple spec with a basic mass matrix and a forcing term.

Spec $(\mathbf{M}, \mathbf{f})$:

\begin{align}
\mathbf{M}\ddot{\mathbf{x}} + \mathbf{f} = \mathbf{0} \\
with: \\
\mathbf{M} &= \begin{pmatrix} 2 & 0 \\ 0 & 0.5 \end{pmatrix} \\
\mathbf{f} &= \begin{pmatrix} x_1 \\ 2 \end{pmatrix}
\end{align}

We can rewrite the above spec equation into a motion policy form of $\ddot{\mathbf{x}} + \mathbf{h}(\mathbf{x}, \mathbf{\dot{x}}) = \mathbf{0}$ as follows: $\ddot{\mathbf{x}} + \mathbf{M}^{-1}\mathbf{f} = \mathbf{0}$.\
This results in the following equation for $\ddot{\mathbf{x}}$:
\begin{align}
\ddot{\mathbf{x}} &= -\mathbf{h} \\
\ddot{\mathbf{x}} &= -\mathbf{M}^{-1}\mathbf{f}
\end{align}

In the next cell, the simple spec is first defined and then rewritten in the motion policy form with concretize.
Feel free to play around with the number in the equation to see the different behaviors.

In [ ]:
x = ca.SX.sym('x', 2)
xdot = ca.SX.sym('xdot', 2)
M = ca.SX([[2.0, 0.0], [0.0, 0.5]])
f = ca.vcat([x[1], 2.0])
simple_spec = Spec(M, f=f, x=x, xdot=xdot)
simple_spec.concretize()

### Planar simulation of simple spec

Test the spec inside a planar simulation environment.\
The spec produces a trajectory which will be visualized by the environment.

In [ ]:
import gym
import numpy as np
import planarenvs.point_robot
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

#### 1. Run the simulation to create the trajectory

In [ ]:
env = gym.make("point-robot-acc-v0", render=True, dt=0.01)
init_pos = np.array([0.0, 0.0])
init_vel = np.array([0.0, 0.0])
ob = env.reset(pos=init_pos, vel=init_vel)
n_steps = 100
positions = np.zeros((n_steps, 2))
for i in range(n_steps):
    x = ob['joint_state']['position']
    positions[i] = x
    xdot = ob['joint_state']['velocity']
    M, h, action = simple_spec.evaluate(x=x, xdot=xdot)
    ob, _, _, _ = env.step(action)
env.close()

#### 2. Visualize the trajectory

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1, 1)
axs.plot(positions[:,0], positions[:,1])
axs.axis('equal')
axs.set_xlim(-5,5)
axs.set_ylim(-5,5)
initial_location = plt.Circle((positions[:,0][0], positions[:,1][0]), radius=0.05, color='y')
axs.add_patch(initial_location)
plt.show()

#### Playing around

Now, play around with the values for the spec and what different behaviors you could get.